In [1]:
import json
import pickle
import random

import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torch.nn.utils.rnn as rnn_utils

from nltk.translate.bleu_score import sentence_bleu
import time

from Vocab import Vocab

import torch
torch.cuda.set_device(1)

print('import over')

import over


In [2]:
def tokenized_sent2real_sent(tokenized_sent, vocab):
    real_sent=[]
    for token in tokenized_sent:
        if token == vocab.word2token['eos']:
            break
        else:
            real_sent.append(vocab.token2word[token])
    return ''.join(real_sent)

def reverse_tokenized_sent2real_sent(tokenized_sent, vocab):
    real_sent=[]
    for token in tokenized_sent:
        if token == vocab.word2token['eos']:
            break
        else:
            real_sent.append(vocab.token2word[token])
    real_sent.reverse()
    return ''.join(real_sent)

def data_set_bleu(sents1, sents2):
    cnt=0
    bleu_score_sum=0
    
    for sent1, sent2 in zip(sents1, sents2):
        if min(len(sent1), len(sent2))<4:
            pass
        else:
            cnt+=1
            bleu_score_sum = sentence_bleu([list(sent1)], list(sent2))+bleu_score_sum
            
    return bleu_score_sum, cnt

with open('vocab.pk', 'rb') as f:
    vocab=pickle.load(f)

In [3]:
print(tokenized_sent2real_sent([5,6,2,1,1], vocab))
print(reverse_tokenized_sent2real_sent([4,5,6,2,1,1], vocab))

公历纷繁复杂
纷繁复杂公历mask


In [4]:
with open('./data_set/train_set_inputs.pk', 'rb') as f:
    train_set_inputs = pickle.load(f)
with open('./data_set/train_set_input_lens.pk', 'rb') as f:
    train_set_input_lens = pickle.load(f)
with open('./data_set/train_set_labels.pk', 'rb') as f:
    train_set_labels = pickle.load(f)
    
with open('./data_set/valid_set_inputs.pk', 'rb') as f:
    valid_set_inputs = pickle.load(f)
with open('./data_set/valid_set_input_lens.pk', 'rb') as f:
    valid_set_input_lens = pickle.load(f)
with open('./data_set/valid_set_labels.pk', 'rb') as f:
    valid_set_labels = pickle.load(f)

In [5]:
print(len(train_set_inputs), len(train_set_input_lens), len(train_set_labels), 
      len(valid_set_input_lens), len(valid_set_inputs), len(valid_set_labels))

for sent_len in valid_set_input_lens:
    if sent_len<=2:
        print('why')

8697000 8697000 8697000 1185955 1185955 1185955


In [6]:
class Encoder(nn.Module):
    def __init__(self, use_cuda, hidden_dim, input_dim, vocab):#, pre_train_weight, is_fix_word_vector = 1):
        super(Encoder, self).__init__()
        
        self.use_cuda = use_cuda
        self.input_dim=input_dim
        self.hidden_dim=hidden_dim
        self.vocab = vocab
        
        self.lstm=torch.nn.LSTM(input_size=self.input_dim, 
                                hidden_size= self.hidden_dim, 
                                bidirectional=True,
                                batch_first=True
                               )
        
        #embedding
        self.embed=nn.Embedding(len(self.vocab.word2token), input_dim)
        
        with open('../pre_train_wordembedding/pre_train_word_embedding.pk', 'rb') as f:
            pre_train_word_embedding = pickle.load(f)
            
        self.embed.weight.data.copy_(torch.FloatTensor(pre_train_word_embedding))
        #self.embed.weight.requires_grad = False
        
#         self.fcnn1=nn.Linear(hidden_dim*2, hidden_dim)
#         self.fcnn2=nn.Linear(hidden_dim, hidden_dim)
#         self.activate_func=nn.ReLU()
        
#         self.loss_func = nn.MSELoss()
        
    def order(self, inputs, inputs_len):
        
        inputs_len, sort_ids = torch.sort(inputs_len, dim=0, descending=True)
        #print(inputs.shape, sort_ids.shape, inputs_len.shape)
        
        if self.use_cuda:
            inputs = inputs.index_select(0, Variable(sort_ids).cuda())
        else:
            inputs = inputs.index_select(0, Variable(sort_ids))
        
        _, true_order_ids = torch.sort(sort_ids, dim=0, descending=False)
        
        return inputs, inputs_len, true_order_ids
    #
    def forward(self, inputs, inputs_len):
        inputs = Variable(inputs)
        if self.use_cuda:
            inputs=inputs.cuda()
            
        inputs, sort_len, true_order_ids = self.order(inputs, inputs_len)

#         print(self.embed.weight.data[0])
#         self.embed.weight.data[0]=torch.LongTensor(([2]*400))
#         print(self.embed.weight.data[0])
#         print(self.embed.weight)
        
        #print('encoder lstm, inputs size:', inputs.size())
        in_vec=self.embed(inputs)
        #print('encoder lstm, in_vec size:', in_vec.size())
        packed = rnn_utils.pack_padded_sequence(input=in_vec, lengths=list(sort_len), batch_first =True)
        
        outputs, (hn,cn) = self.lstm(packed)
        outputs, sent_lens = rnn_utils.pad_packed_sequence(outputs)
        
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        
        outputs = outputs.transpose(0,1)
        #print(outputs)
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        
        if self.use_cuda:
            outputs = outputs.index_select(0, Variable(true_order_ids).cuda())
        else:
            outputs = outputs.index_select(0, Variable(true_order_ids))
        
        #print('outpurs size, hn size and cn size: ', outputs.size(), hn.size(), cn.size())
        #
        # warnning: hn and cn have not been sorted by sentences length so the order is wrong.
        #
        #print('hn and cn: ', hn, cn)
        hn = torch.cat((hn[0], hn[1]), dim=1)
        cn = torch.cat((cn[0], cn[1]), dim=1)
        #print('hn size and cn size: ', hn.size(), cn.size())
        
        if self.use_cuda:
            hn = hn.index_select(0, Variable(true_order_ids).cuda())
            cn = cn.index_select(0, Variable(true_order_ids).cuda())
        else:
            hn = hn.index_select(0, Variable(true_order_ids))
            cn = cn.index_select(0, Variable(true_order_ids))
            
        #print('hn and cn: ', hn, cn)
        
#         print(outputs.size())
#         print(hn.size())

        return outputs, (hn,cn)

In [7]:
class Decoder(nn.Module):
    def __init__(self, use_cuda, encoder, hidden_dim, max_length=25):
        super(Decoder, self).__init__()
        
        self.use_cuda = use_cuda
        self.hidden_dim=hidden_dim
        self.input_dim = encoder.input_dim
        self.max_length = max_length
        self.vocab = encoder.vocab
        self.weight = [1]*len(self.vocab.word2token)
        self.weight[self.vocab.word2token['padding']]=0
        #self.weight[self.vocab.word2token['eos']]=1.01
        
#         self.lstm=torch.nn.LSTM(input_size=self.input_dim, 
#                                 hidden_size= self.hidden_dim, 
#                                 bidirectional=True,
#                                 batch_first=True
#                                )
        
        self.lstm = torch.nn.LSTMCell(input_size=self.input_dim, hidden_size=self.hidden_dim*2, bias=True)
        
        #embedding
        self.embed=encoder.embed# reference share
        #softmax
        self.fcnn = nn.Linear(in_features = self.hidden_dim*2, out_features = len(self.vocab.word2token))
        
#         with open('../pre_train_wordembedding/pre_train_word_embedding.pk', 'rb') as f:
#             pre_train_word_embedding = pickle.load(f)
#         self.fcnn.weight.data.copy_(torch.FloatTensor(pre_train_word_embedding))
#         self.fcnn.weight.requires_grad = True
#         #self.fcnn.bias.data.fill_(0.0)
#         #self.fcnn.bias.requires_grad = True
        
#         self.transform = nn.Linear(in_features = self.hidden_dim*2, out_features = self.input_dim)
#         #self.transform.bias.data.fill_(0.0)
#         #self.transform.bias.requires_grad = False
        
        self.softmax = nn.Softmax()
        self.cost_func = nn.CrossEntropyLoss(torch.Tensor(self.weight))
        
        print('init lookup embedding matrix size: ', self.embed.weight.data.size())
        
    def forward(self, enc_outputs, sent_lens, h0_and_c0, labels, teaching_rate=0.6, is_train=1, force_argmax=1):
        labels = Variable(labels)
        if self.use_cuda:
            labels = labels.cuda()
            

        all_loss = 0
        predicts = []
        batch_size = enc_outputs.size(dim = 0)

        
        #print('enc_outputs size: ', enc_outputs.size())
#         for ii in range(batch_size):
#             forward_result = enc_outputs[ii, sent_lens[ii]-1, :self.hidden_dim].unsqueeze(dim=0)
#             reverse_result = enc_outputs[ii, 0, self.hidden_dim:].unsqueeze(dim=0)
#             final_hidden_states[ii] = torch.cat([forward_result, reverse_result], dim = 1)   #so enc_outputs[ii, sent_lens[ii]-1, :] 's size: self.hidden_dim*2
        
#         #print('final_hidden_states[0] size', final_hidden_states[0].size())
#         final_hidden_states = torch.cat(final_hidden_states, dim=0)
        
        #print('final_hidden_states size: ', final_hidden_states.size())
        
        final_hidden_states = h0_and_c0[0]
        embed_weight = self.embed.weight
        embed_weight_expand  =embed_weight.expand(batch_size, embed_weight.size(0), embed_weight.size(1))
        
        for ii in range(self.max_length+1):
            if ii==0:
                zero_timestep_input = torch.LongTensor([self.vocab.word2token['sos']]*batch_size)
                zero_timestep_input = Variable(zero_timestep_input)
                if self.use_cuda:
                    zero_timestep_input = zero_timestep_input.cuda()
                    
                zero_timestep_input = self.embed(zero_timestep_input)#size: batch_size * self.input_dim

                #print('zero_timestep_input size: ', zero_timestep_input.size())
                
                
#                 h0 = Variable(torch.zeros((batch_size, self.hidden_dim*2)))
#                 c0 = Variable(torch.zeros((batch_size, self.hidden_dim*2)))
#                 if self.use_cuda:
#                     h0 = h0.cuda()
#                     c0 = c0.cuda()
                    
                #h0_and_c0=(h0,c0)
                
                #print('zero_timestep_input size: ', zero_timestep_input.size())
                #print('final_hidden_states size: ', final_hidden_states.size())
                
                #last_timestep_hidden_state,cx = self.lstm(torch.cat([zero_timestep_input, final_hidden_states],dim=1), h0_and_c0)#h0 and c0
                
                last_timestep_hidden_state,cx = self.lstm(zero_timestep_input, h0_and_c0)
                #print('hn and cn sizes: ', last_timestep_hidden_state.size(), cx.size())
                
                #last_timestep_hidden_state,cx = self.lstm(zero_timestep_input,(h0_and_c0[0]-h0_and_c0[0], h0_and_c0[1]-h0_and_c0[1]))
                
                #print(last_timestep_hidden_state.size(), type(hn), type(cn))
                #last_timestep_output = self.softmax(self.fcnn(last_timestep_hidden_state))
                last_timestep_output = self.fcnn(last_timestep_hidden_state)
                
                #print('last_timestep_hidden_state size: ', last_timestep_output.size(), 'last_timestep_output size: ', last_timestep_output.size())
                if is_train:
                    #print('last_timestep_output size: ', last_timestep_output.size(), labels[:,0].size())
                    loss = self.cost_func(last_timestep_output, labels[:,0])
                    #print('loss size: ', loss.size(), loss.data, torch.mean(loss).data)
                    all_loss+=loss
                #else:
                #print(last_timestep_output.size())
                #x=torch.squeeze(last_timestep_output, dim=1)
                #print('x size: ', x.size())
                _, max_idxs = torch.max(last_timestep_output, dim=1)
                predicts.append(torch.unsqueeze(max_idxs, dim=0))
                #print('max_idxs size: ',max_idxs.size(), max_idxs)
                
            else: # ii !=0
                if is_train:
                    rand = random.random()
                    if rand<teaching_rate:
                        #print('rand: ', rand)
                        this_timestep_input = self.embed(labels[:,ii-1])#label teaching, lookup embedding
                    elif force_argmax==1:
                        this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                    else:
                        
                        #print(embed_weight.size(), last_timestep_output.size())
                        this_timestep_input = torch.bmm(last_timestep_output.unsqueeze(dim=1), 
                                                embed_weight_expand)#global embedding
                        #print('block multi', this_timestep_input.size())
                else:
                    if force_argmax==1:
                        this_timestep_input = self.embed(max_idxs)#last_timestep output, and then look up word embedding
                    else:
                        
                        #print(embed_weight.size(), last_timestep_output.size())
                        this_timestep_input = torch.bmm(last_timestep_output.unsqueeze(dim=1), 
                                                embed_weight_expand)#global embedding
                        #print('block multi', this_timestep_input.size())
                
                
                #print('this_timestep_input size: ', this_timestep_input.size())
                #print('final_hidden_states size: ', final_hidden_states.size())
#                 last_timestep_hidden_state ,cx = self.lstm(torch.cat([this_timestep_input.squeeze(dim=1), final_hidden_states],dim=1)
#                                                            , (last_timestep_hidden_state,cx))

                last_timestep_hidden_state ,cx = self.lstm(this_timestep_input, (last_timestep_hidden_state,cx))

                #last_timestep_output = self.softmax(self.fcnn(last_timestep_hidden_state))
                last_timestep_output = self.fcnn(last_timestep_hidden_state)
                
                if is_train:
                    #print('x size and labels[:, 0] size: ', x.size(), labels[:,0].size())
                    #x=torch.squeeze(last_timestep_output, dim=1)
                    #print('x size and labels[:, 0] size: ', x.size(), labels[:,0].size())
                    loss = self.cost_func(last_timestep_output, labels[:,ii])
                    all_loss+=loss
                
                #x=torch.squeeze(last_timestep_output, dim=1)
                _, max_idxs = torch.max(last_timestep_output, dim=1)
                #print('max_idx size: ', max_idxs.size(), max_idxs)
                predicts.append(torch.unsqueeze(max_idxs, dim=0))
                
#                 if ii==1:
#                     print('this_timestep_input size: ', this_timestep_input.size(), 'max_idxs size: ', max_idxs.size())
                    
#         if is_train:
#             return all_loss/(self.max_length+1)
#         else:
#             #predict results
#             predicts = torch.cat(predicts, dim=0)
#             #print('predicts size: ', predicts.size())
#             predicts = torch.transpose(predicts, 0, 1)
#             if self.use_cuda:
#                 return predicts.data.cpu().numpy()
#             else:
#                 return predicts.data.numpy()


        predicts = torch.cat(predicts, dim=0)
        predicts = torch.transpose(predicts, 0, 1)
        #print('predicts size: ', predicts.size())
        
        if is_train:  #training
            if self.use_cuda:
                return all_loss/(self.max_length+1), predicts.data.cpu().numpy()
            else:
                return all_loss/(self.max_length+1), predicts.data.numpy()
        else:   #testing
            if self.use_cuda:
                return predicts.data.cpu().numpy()
            else:
                return predicts.data.numpy()
            

                

In [8]:
use_cuda = 1
hidden_dim = 512
input_dim = 300


enc = Encoder(use_cuda=use_cuda, 
            hidden_dim=hidden_dim, 
            input_dim=input_dim, 
            vocab=vocab
           )
if use_cuda:
    enc = enc.cuda()
    
    
sample_num = 11
print('sentences length: ', train_set_input_lens[0:sample_num])

enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(train_set_inputs[0:sample_num]), 
                                    torch.LongTensor(train_set_input_lens[0:sample_num]))
print('enc result size: ', enc_outputs.size(), enc_hn.size(), enc_cn.size())

dec = Decoder(use_cuda=use_cuda, encoder=enc, hidden_dim=hidden_dim, max_length=25)
if use_cuda:
    dec = dec.cuda()
    
loss, predicts = dec(enc_outputs = enc_outputs, 
    h0_and_c0=(enc_hn, enc_cn), 
    sent_lens=train_set_input_lens[0:sample_num], 
    labels=torch.LongTensor(train_set_labels[0:sample_num]), 
    is_train=1, teaching_rate = 1, force_argmax=1)
print('loss is %4.7f'%loss.data[0])
# print(enc_outputs, enc_hn, enc_cn)
#print(enc_hn, enc_cn)

# pre_trained = torch.load('./models_saved/dec-loss-8.221987724-bleu-0.0000-hidden_dim-256-input_dim-256-lr-0.0050') 
# dec.load_state_dict(pre_trained)

# pre_trained = torch.load('./models_saved/enc-loss-8.221987724-blue-0.0000-hidden_dim-256-input_dim-256-lr-0.0050') 
# enc.load_state_dict(pre_trained)

# enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(train_set_inputs[0:sample_num]), 
#                                     torch.LongTensor(train_set_input_lens[0:sample_num]))
# dec(enc_outputs = enc_outputs, 
#     h0_and_c0=(enc_hn, enc_cn), 
#     sent_lens=train_set_input_lens[0:sample_num], 
#     labels=torch.LongTensor(train_set_labels[0:sample_num]), 
#     is_train=1, teaching_rate = 1, force_argmax=1)

sentences length:  (8, 11, 11, 13, 7, 9, 25, 7, 7, 11, 8)
enc result size:  torch.Size([11, 25, 1024]) torch.Size([11, 1024]) torch.Size([11, 1024])
init lookup embedding matrix size:  torch.Size([98637, 300])
loss is 11.4964514


In [11]:
lr=0.005

epochs=500000
start_idx=0
batch_size=200
train_set_size=int(len(train_set_inputs)/2)

model_paras = list(dec.parameters())+list(enc.parameters())
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_paras), lr=lr)#optimizer for epoch in range(epochs):

start_time = time.time()

bleu_score = 0

for epoch in range(epochs):
    #print(epoch)

    optimizer.zero_grad()#clear

    enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(train_set_inputs[start_idx:start_idx+batch_size]), 
                                        torch.LongTensor(train_set_input_lens[start_idx:start_idx+batch_size]))

    loss, predicts = dec(enc_outputs, h0_and_c0=(enc_hn, enc_cn), 
               sent_lens = train_set_input_lens[start_idx:start_idx+batch_size],
               labels=torch.LongTensor(train_set_labels[start_idx:start_idx+batch_size]), 
               teaching_rate=1,
               is_train=1,
               force_argmax = 1)
    #optimize
    loss.backward()#retain_graph=True)
    optimizer.step()

    start_idx=epoch*batch_size%(train_set_size-batch_size-1)    # samples change

    
    valid_batch_size=10
    valid_start_idx=random.randint(0, train_set_size-valid_batch_size-1)
    if random.random()<0.5:
        valid_start_idx=random.randint(0, len(train_set_inputs)-1-valid_batch_size-1)
    #valid_start_idx=random.randint(0, train_set_size-valid_batch_size-1)
    
    if epoch%50 == 1:
        
        enc.eval()
        dec.eval()
        enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(train_set_inputs[valid_start_idx:valid_start_idx+valid_batch_size]), 
                                        torch.LongTensor(train_set_input_lens[valid_start_idx:valid_start_idx+valid_batch_size]))

        loss_, predicts = dec(enc_outputs, 
                       sent_lens = train_set_input_lens[valid_start_idx:valid_start_idx+valid_batch_size],
                       h0_and_c0=(enc_hn, enc_cn), 
                       labels=torch.LongTensor(train_set_labels[valid_start_idx:valid_start_idx+valid_batch_size]),
                       is_train=1,
                       teaching_rate=1,
                       force_argmax=1)
        del loss_

        tokenized_sents=predicts.tolist()
        real_sents=[]
        label_tokenized_sents=train_set_labels[valid_start_idx:valid_start_idx+valid_batch_size]
        label_real_sents=[]
        for idx, sent in enumerate(tokenized_sents):
            real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))

        for sent in label_tokenized_sents:
            label_real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))

        print('valid_start_idx: ', valid_start_idx)
        for (real_sent, label_real_sent) in zip(real_sents, label_real_sents):
            print(real_sent, 'end           ', label_real_sent)
        
        
        
        #valid
        if epoch%1000==1:
            valid_rand_idx=random.randint(0, len(valid_set_inputs)-batch_size-1-1)
            enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(valid_set_inputs[valid_rand_idx:valid_rand_idx+batch_size]), 
                                            torch.LongTensor(valid_set_input_lens[valid_rand_idx:valid_rand_idx+batch_size]))

            predicts = dec(enc_outputs, 
                           sent_lens = valid_set_input_lens[valid_rand_idx:valid_rand_idx+batch_size],
                           h0_and_c0=(enc_hn, enc_cn), 
                           labels=torch.LongTensor([0]), 
                           is_train=0,
                           teaching_rate=1,
                           force_argmax=1)
            
            tokenized_sents=predicts.tolist()
            real_sents=[]
            label_tokenized_sents=valid_set_labels[valid_rand_idx:valid_rand_idx+batch_size]
            label_real_sents=[]
            for sent in tokenized_sents:
                real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))
                #real_sents.append(reverse_tokenized_sent2real_sent(sent, enc.vocab))
            for sent in label_tokenized_sents:
                label_real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))
                #label_real_sents.append(reverse_tokenized_sent2real_sent(sent, enc.vocab))

            bleu_sum, valid_num = data_set_bleu(label_real_sents, real_sents)
            if valid_num>10:
                bleu_score = bleu_sum/valid_num
            else:
                bleu_score = 999
            #sava
            torch.save(enc.state_dict(), 
                    './models_saved/enc-loss-{:2.9f}-bleu-{:1.4f}-hidden_dim-{:n}-input_dim-{:n}-lr-{:1.4f}'.format(
                          loss.data[0], bleu_score, hidden_dim, input_dim, lr))
            torch.save(dec.state_dict(),
            './models_saved/dec-loss-{:2.9f}-bleu-{:1.4f}-hidden_dim-{:n}-input_dim-{:n}-lr-{:1.4f}'.format(
                    loss.data[0], bleu_score, hidden_dim, input_dim, lr))

        print('%2.1f%%(%s/%s), loss: %2.9f, bleu_score: %1.4f'%(epoch*100/epochs, epoch, epochs, loss.data[0], bleu_score))
        
        enc.train()
        dec.train()
        
print('running time: %.2f mins'%((time.time()-start_time)/60))

valid_start_idx:  3493882
我觉得low_freqlow_freq，的的，多的low_freq。。 end            我和所罗门兄弟公司有过一次不太愉快的合作经历。
在low_freq里low_freq，、，的，，low_freqlow_freq、免费、冰箱、、、免费、免费、免费冰箱 end            从享用免费欧式早餐开始精力充沛的一天，包括华夫饼、热咖啡、热腾腾的茶水、酥饼和面包、新鲜水果
他们是是这里里。 end            他们的确在丛林里。
快他们的是是的的的。 end            告诉他们。这是不对的。
有一个人的人呢？ end            以一个新的身份潜逃？
我觉得了，伙计。 end            我挂了，拜。
她的，，我们，我的，我们还在在并它的让它们的low_freq提供，中国年代， end            更糟的是，巴茨加入雅虎时，公司董事会承诺，如果她能够让公司的股票价格回升到25美元/
但我想，，，会地。 end            因为我开车的时候我们不停接吻。
他们不，你事 end            并不意味着什么坏事
你想想你你你就知道 end            你刚刚到这儿难怪你不知道
0.0%(1/500000), loss: 3.864894390, bleu_score: 0.4393
valid_start_idx:  1532679
你知道去你的衣服。 end            你不能离开你的妻子。
是啊，我是是在在他们没有了。 end            是啊，本来我还不确定是不是定了。
他的是都low_freq。 end            他的家里满是这些。
现在你是，，我是是了个！ end            现在你知道了，你也有了犯罪感！
所以我的在我的我的的情景。 end            所以我想是因为面包和果酱low_freq的关系。
low_freq人的的是的的的在是在的很久。 end            这些新殖民主义者不仅压迫第三世界人民，也会压迫本国公民。
你说说吗？好明白。 end            你不会介意吧？当然不。
孩子！，low_freq了。。 end        

KeyboardInterrupt: 

In [ ]:
stop_this_cell

#print('running time: %.2f mins'%((time.time()-start_time)/60))
#sava
torch.save(enc.state_dict(), 
        './models/enc-loss-{:2.9f}-blue-{:1.4f}-hidden_dim-{:n}-input_dim-{:n}-lr-{:1.4f}'.format(
              loss.data[0], bleu_score, hidden_dim, input_dim, lr))
torch.save(dec.state_dict(),
'./models/dec-loss-{:2.9f}-bleu-{:1.4f}-hidden_dim-{:n}-input_dim-{:n}-lr-{:1.4f}'.format(
        loss.data[0], bleu_score, hidden_dim, input_dim, lr))
#a little ok: dec-loss-8.243618965-bleu-0.0000-hidden_dim-256-input_dim-256-lr-0.0050 and enc-......
#results： for example：
# 我对我们一上，上一里里不对里。             全给我吃完，地上一粒米也不许剩。
# 有能和我，我能对。她这里。              喝得烂醉；我只能把她赶出去。 
# 了，这是吗？。 是的。对吗。              是这样吗？ 是的。随便啦。 
# 是你的！是！！来！！来！来！来！              你喝的都是自来水！去找饮用水来！ 
# 以他以不是这里的。              他以前不是那样的。 
#
print('loss: %2.9f'%loss.data[0])

In [ ]:
valid_start_idx

In [ ]:
valid_start_idx=600
valid_batch_size=30

dec.eval()
enc.eval()
enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(valid_set_inputs[valid_start_idx:valid_start_idx+valid_batch_size]), 
                                torch.LongTensor(valid_set_input_lens[valid_start_idx:valid_start_idx+valid_batch_size]))

predicts = dec(enc_outputs, 
               sent_lens = valid_set_input_lens[valid_start_idx:valid_start_idx+valid_batch_size],
               h0_and_c0=(enc_hn, enc_cn), 
               labels=torch.LongTensor([0]), 
               is_train=0,
               teaching_rate=0,
               force_argmax = 1)

tokenized_sents=predicts.tolist()
real_sents=[]
label_tokenized_sents=valid_set_labels[valid_start_idx:valid_start_idx+valid_batch_size]
label_real_sents=[]
for sent in tokenized_sents:
    real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))
for sent in label_tokenized_sents:
    label_real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))

for (real_sent, label_real_sent) in zip(real_sents, label_real_sents):
    print(real_sent, '           ', label_real_sent)
    
    
print('### ........................ ###')
print('### ........................ ###')
print('### ........................ ###')


valid_start_idx=620
valid_batch_size=200

dec.eval()
enc.eval()
enc_outputs, (enc_hn, enc_cn) = enc(torch.LongTensor(valid_set_inputs[valid_start_idx:valid_start_idx+valid_batch_size]), 
                                torch.LongTensor(valid_set_input_lens[valid_start_idx:valid_start_idx+valid_batch_size]))

predicts = dec(enc_outputs, 
               sent_lens = valid_set_input_lens[valid_start_idx:valid_start_idx+valid_batch_size],
               h0_and_c0=(enc_hn, enc_cn), 
               labels=torch.LongTensor([0]), 
               is_train=0,
               teaching_rate=1,
               force_argmax = 1)

tokenized_sents=predicts.tolist()
real_sents=[]
label_tokenized_sents=valid_set_labels[valid_start_idx:valid_start_idx+valid_batch_size]
label_real_sents=[]
for sent in tokenized_sents:
    real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))
for sent in label_tokenized_sents:
    label_real_sents.append(tokenized_sent2real_sent(sent, enc.vocab))

# for (real_sent, label_real_sent) in zip(real_sents, label_real_sents):
#     print(real_sent, '           ', label_real_sent)
    
dec.train()
enc.train()

In [ ]:
def data_set_bleu1(sents1, sents2):
    cnt=0
    bleu_score_sum=0
    
    for sent1, sent2 in zip(sents1, sents2):
        if min(len(sent1), len(sent2))<4:
            pass
        else:
            cnt+=1
            bleu_score_sum = sentence_bleu([list(sent1)], list(sent2))+bleu_score_sum
            
    return bleu_score_sum, cnt

bleu_sum, valid_num = data_set_bleu1(label_real_sents, real_sents)
print(len(label_real_sents), bleu_sum, valid_num, bleu_sum/valid_num)
